# Implicit Ratings
Based on users behaviour rather than explicit ratings. For example, if a person clicks on links about technology but never about sports on a news site, that tells us something about their preferences. Another example would be keeping track of things people have bought - Amazon even has a 'frequently bought together' section on the site once you put something in your cart.

Problems with implicit ratings:
- People don't bother rating items
- People lie
- People don't update ratings

One of the tricky things about implicit ratings is that if someone buys a gift, it can confuse your system. You want to try and build up a profile of the user so that your recommendations fit their preferences.

## Item Based Filtering
So far we've been doing user-based collaborative filtering, but this is difficult to scale and results in sparse datasets. Instead, it might be better to do item-based filtering. Here we identify products that are most similar to each other.

For example: let's find the cosine similarity between song 1 and song 2. To do this, you'll need to find all the instances where people have rated both songs. We can calculate the similarity using adjusted cosine similarity

## Adjusted Cosine Similarity
To adjust for inflation of ratings by users, we'll subtract the average rating of each user from each of their ratings. This equation shows the similarity between items i and j. R is the rating, u is a user. $\bar{R}_u$ is the average rating the user gave all items that they rated 

$$
s(i, j) = \frac{\sum_{u \in U}(R_{u,i} - \bar{R}_u)(R_{u,j} - \bar{R}_u)}
{\sqrt{\sum_{u \in U}(R_{u,i} - \bar{R}_u)^2} \sqrt{\sum_{u \in U}(R_{u,j} - \bar{R}_u)^2}}
$$

In [27]:
import math
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

In [28]:
reviews = { 'users': ['Tamera', 'Jasmine', 'Arturo', 'Cecilia', 'Jesssica', 'Jordyn'],
            'average_rating': [3.5, 3, 3, 3.5, 4, 2.5],
            'song1': [5, np.nan, 1, 5, 4, 4],
            'song2': [np.nan, 4, 2, 5, 5, np.nan]}
df = pd.DataFrame(reviews)
df

,average_rating,song1,song2,users
0,3.5,5.0,NaN,Tamera
1,3.0,NaN,4.0,Jasmine
2,3.0,1.0,2.0,Arturo
3,3.5,5.0,5.0,Cecilia
4,4.0,4.0,5.0,Jesssica
5,2.5,4.0,NaN,Jordyn


In [29]:
# only include columns where both songs were rated by user
df = df.dropna()

In [47]:
minus_average_squared = sum([math.pow((s1 - average) * (s2 - average), 2) for average, s1, s2 in zip(df['average_rating'], df['song1'], df['song2'])])
cs =    (( sum([((s1 - average) * (s2 - average)) for average, s1, s2 in zip(df['average_rating'], df['song1'], df['song2'])]) ) /
        ( math.sqrt(sum([math.pow(s1 - average, 2) for average, s1, s2 in zip(df['average_rating'], df['song1'], df['song2'])])) * 
          math.sqrt(sum([math.pow(s2 - average, 2) for average, s1, s2 in zip(df['average_rating'], df['song1'], df['song2'])])) ))
print(cs)

0.824621125124


## Weighted Slope One
Very simple algorithm to predict ratings. If Amy rated Psy 3 and Whitney Houston 4, and Ben rated Psy 4, what should we predict Ben's rating of Whitney Housten would be? Well since Amy liked Whitney Housten 1 more than Psy, we could get that Ben's rating would be 5. Slope 1 is conducted in two parts:

1. Ahead of time in batch mode - you calculate the deviation between every pair of items
2. Now you have the list of deviations, you can predict how they'd rate something they haven't seen using things they have rated.

Calculating the deviations for item i with respect to item j. card(S) is how many elements are in S and X is the entire set of rankings. $card(S_{i,j}(X))$ is the ttoal number of people who rated both i and j

$$
dev_{i,j} = \sum_{u \in S_{i,j}} \frac{u_i - u_j}{card(S_{i,j}(X))}
$$

In [71]:
ratings = {'Taylor Swift': [4,5,np.nan,5], 'Psy': [3,2,3.5,np.nan], 'Whitney Houston': [4,np.nan,4,3]}
df = pd.DataFrame(ratings)
df

,Psy,Taylor Swift,Whitney Houston
0,3.0,4.0,4.0
1,2.0,5.0,NaN
2,3.5,NaN,4.0
3,NaN,5.0,3.0


## 1. Calculating Slope Deviations

In [81]:
dfs = df[['Psy', 'Taylor Swift']].dropna()
# calculate deviation for Psy with respect to Taylor Swift
psy = sum([((a1 - a2) / len(dfs['Psy'])) for a1, a2 in zip(dfs['Taylor Swift'], dfs['Psy'])])
# calculate the reverse: Taylor Swift with respect to Psy
swift = sum([((a1 - a2) / len(dfs['Psy'])) for a1, a2 in zip(dfs['Psy'], dfs['Taylor Swift'])])
print(psy, swift)
print('The average user rated Swift 2 points higher than Psy')

2.0 -2.0
The average user rated Swift 2 points higher than Psy


## 2. Making Predictions with Slope
Let's make a prediction about what Ben would rate Whitney Houston.

$$
P^{wS1}(u)_j = \frac{\sum_{i \in S(u)-\{j\}}(dev_{j,i} + u_i)c_{j,i}}
{\sum_{i \in S(u)-\{j\}}c_{j,i}}
$$

- In this equation, $c_{j,i} = card((S_{j,i}(X))$
- $P^{wS1}(u)_j$ is our prediction using user u's rating for item j
- $\sum_{i \in S(u)-\{j\}}$ Means for every musician Ben has rated (except for Houston - {j}), we look up the deviation of Whitney Houston to the musician and will add that to Ben's rating for musician i. We multiply it by the cardinality of that pair - the number of people that rated both Whitney and musician i.



| Ben's ratings  | Taylor Swift  | PSY  | Whitney Houston  |
|:---:|:---:|:---:|:---:|
|  Ben |  5 | 2  |  ? |



| Deviations  | Taylor Swift  |  PSY |  Whitney Houston |
|:---:|:---:|:---:|:---:|
| Taylor Swift  |  0 | 2  | 1  |
| PSY  |  -2 |  0 | -0.75  |
| Whitney Houston  | -1  | 0.75  |  0 |

- $u_i$ is Ben's rating of 5 for Taylor Swift and $dev_j,i$ is the deviation of Whitney Houston compared to Swift: -1. Result: 4
- Two people rated both Swift and Houston, so $c_{j,i}$ is 2
- Result: $(dev_{j,i} + u_i) c_{j,i} = 4 x 2 = 8$
- Ben rated Psy and gave him 2: $(dev_{j,i} + u_i) c_{j,i} = 2.75 x 2 = 5.5$
- Sum up result for both and get 13.5 for the numerator.
- The denominator looks at how for every musician Ben rated, we sum the cardinalities of those musicians (how many people rated both that musician and Whitney Houston)
- So Ben rated Swift and the cardinality of Swift and Houston (total number of people that rated them both) is 2. Ben rated Psy and his cardinality is 2. Therefore the denominator is 4
- Predicted rating of Houston is 13.5/4 = 3.375

$$

$$

-2.0